# Demonstração Operacional do Protocolo de Integridade Digital via Cifra de Integridade Primal (CIP)

**Registro operacional associado ao livro**  
*Descobrindo o Caos nos Números — Como a ordem emerge quando mudamos a forma de observar*  
© Alvaro Costa, 2025  

Este notebook integra a sequência canônica de **registros computacionais da Cifra de Integridade Primal (CIP)**.

Ele **não introduz hipóteses, conjecturas, modelos matemáticos ou argumentos conceituais novos**.  
Seu papel é exclusivamente operacional.

O objetivo deste registro é **demonstrar, por execução direta**, o funcionamento dos verbos do protocolo CIP  
em um ambiente controlado, determinístico e reprodutível.

A fundamentação matemática, conceitual e epistemológica do protocolo encontra-se integralmente desenvolvida no livro.  
Este notebook não interpreta resultados, não defende o protocolo e não substitui a leitura teórica.

Ele **registra**.

As operações aqui executadas — assinatura, verificação, cifragem, decifragem e interoperabilidade estrutural —  
devem produzir resultados observáveis e auditáveis, independentes de plataforma, ambiente ou confiança institucional.

Se o protocolo for correto, ele se sustenta por execução.  
Se não for, nenhuma formulação teórica o resgata.

**Licença:** Creative Commons BY–NC–ND 4.0  

É permitida a leitura, execução e citação acadêmica.  
Não é permitida a modificação do conteúdo, redistribuição adaptada ou uso comercial independente.


## Demonstração reprodutível no Google Colab: 
### chaves externas (IDX 18 / IDX 20)
### verificadas por um único binário local (IDX 0)

Este notebook demonstra, de forma pública e reprodutível no Google Colab, que o **PID** — baseado na **Cifra de Integridade Primal (CIP)** — mantém **interoperabilidade estrutural** entre chaves emitidas por SDKs distintos, associados a **índices espectrais públicos (IDX)** diferentes.

O experimento parte de um cenário realista: **SDKs já distribuídos**, **chaves já emitidas** e **um ambiente neutro de verificação** (Colab).
O notebook não recompila SDKs e não gera novas assinaturas. Ele apenas verifica, de modo autocontido, a consistência do protocolo.

### Artefatos utilizados

Trabalhamos com **duas chaves CIP-T** (modo TOTAL, `-t`) geradas a partir do **mesmo arquivo**:

* **Chave A**: emitida por um SDK Linux **IDX 18**
* **Chave B**: emitida por um SDK Windows **IDX 20**

A verificação é executada por **um único binário CIP local**, nativo do Colab (Linux), compilado com **IDX 0**.

### O que esta demonstração comprova

1. O **mesmo arquivo**, assinado por SDKs distintos (**IDX 18** e **IDX 20**), produz **chaves diferentes** (porque cada IDX define uma base harmônica própria).
2. O **mesmo binário CIP local** (Linux/Colab, **IDX 0**) é capaz de:

   * validar o arquivo usando a chave emitida no Linux (**IDX 18**);
   * validar o arquivo usando a chave emitida no Windows (**IDX 20**).
3. A decisão de integridade **não depende** do ambiente que gerou a assinatura:

   * não depende do sistema operacional (Linux vs Windows),
   * não depende do compilador,
   * não depende do **IDX do verificador local**.
4. Ao introduzir uma corrupção mínima no arquivo (alteração de 1 bit), **ambas** as chaves deixam de validar, cada uma sob sua própria régua espectral.
5. A interoperabilidade decorre de um mecanismo explícito do protocolo: quando a chave é **EXTERNA** ao SDK local, a verificação é realizada com a **matriz espectral embutida na própria chave**, tornando o verificador local um executor determinístico do padrão — e não um árbitro dependente de ambiente.

Em síntese: no PID/CIP, a integridade é uma propriedade do **par (arquivo, chave)** sob a régua espectral descrita na chave — e não do sistema que executa a verificação.

## CÉLULA 1 — Imports, detecção de ambiente e configuração básica

Nesta célula preparamos o ambiente do experimento.  
O objetivo aqui **não é compilar nada**, mas apenas:

* detectar o sistema operacional do runtime (Colab → Linux),

* definir uma estrutura de diretórios clara e isolada,

* importar utilitários que serão usados ao longo do notebook.

Todo o experimento é conduzido a partir de **artefatos já existentes**  
(binário CIP local + chaves externas).

In [1]:
# ============================================================
# CÉLULA 1 — Configuração básica de ambiente
# ============================================================

import os
import stat
import shutil
import zipfile
import subprocess
import time
import random

from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# ------------------------------------------------------------
# Diretório base do notebook
# ------------------------------------------------------------
# No Colab, este será tipicamente /content
BASE_DIR = Path.cwd()
print(f"[INFO] Diretório base do notebook: {BASE_DIR}")

# ------------------------------------------------------------
# Detecção simples de sistema operacional
# ------------------------------------------------------------
# (Importante apenas para portabilidade do notebook;
#  neste experimento, o runtime efetivo é Linux.)
WINDOWS = os.name == "nt"
print(f"[INFO] Sistema detectado: {'Windows' if WINDOWS else 'Linux'}")

# ------------------------------------------------------------
# Estrutura de diretórios do experimento
# ------------------------------------------------------------
# sdk_local     → binário CIP usado para VERIFICAÇÃO
# chaves_interop→ chaves CIP-T externas (Linux IDX 18 / Windows IDX 20)
# arquivos      → arquivo alvo da verificação
# resultados    → tabelas, resumos e artefatos gerados no notebook
SDK_DIR        = BASE_DIR / "sdk_local"
CHAVES_DIR     = BASE_DIR / "chaves_interop"
ARQUIVOS_DIR   = BASE_DIR / "arquivos"
RESULTADOS_DIR = BASE_DIR / "resultados"

for d in [SDK_DIR, CHAVES_DIR, ARQUIVOS_DIR, RESULTADOS_DIR]:
    d.mkdir(exist_ok=True)
    print(f"[OK] Diretório garantido: {d}")

# ------------------------------------------------------------
# Limite de segurança para arquivos analisados
# ------------------------------------------------------------
# O foco aqui é integridade estrutural, não stress de I/O.
LIMITE_BYTES = 1_000_000_000  # 1 GB
print(f"[INFO] Limite de tamanho de arquivo: {LIMITE_BYTES/1e6:.1f} MB")


[INFO] Diretório base do notebook: /home/alvaro/descobrindo_caos
[INFO] Sistema detectado: Linux
[OK] Diretório garantido: /home/alvaro/descobrindo_caos/sdk_local
[OK] Diretório garantido: /home/alvaro/descobrindo_caos/chaves_interop
[OK] Diretório garantido: /home/alvaro/descobrindo_caos/arquivos
[OK] Diretório garantido: /home/alvaro/descobrindo_caos/resultados
[INFO] Limite de tamanho de arquivo: 1000.0 MB


## CÉLULA 2 — Download (ou uso local) do SDK nativo

Nesta célula preparamos **um único binário CIP local**, que será utilizado exclusivamente como verificador ao longo de todo o notebook.

Pontos importantes:

* Este notebook **não recompila SDKs**;

* O binário local **não precisa corresponder** aos SDKs que geraram as chaves;

* A interoperabilidade demonstrada **não depende** de qual SDK é “nativo” do ambiente;

* O papel deste binário é apenas executar o protocolo de verificação.

Para efeito de reprodutibilidade no Colab, utilizamos um **SDK Linux já empacotado**.

In [2]:
# ============================================================
# CÉLULA 2 — Download e preparação do binário CIP (Linux)
# ============================================================

import os
import stat
import zipfile
import requests
from pathlib import Path

# ------------------------------------------------------------
# Diretórios base
# ------------------------------------------------------------
BASE_DIR = Path.cwd()
SDK_DIR = BASE_DIR / "sdk_local"
SDK_DIR.mkdir(exist_ok=True)

# ------------------------------------------------------------
# Artefato único do experimento
# ------------------------------------------------------------
# Este binário será usado APENAS como verificador local.
# A interoperabilidade demonstrada não depende do seu IDX.
ZIP_NAME = "cip-sdk-linux-idx0.zip"
SDK_FOLDER_NAME = "cip-sdk-linux-idx0"
DOWNLOAD_URL = "https://api.delta-cip.com.br/downloads/colab/cip-sdk-linux-idx0.zip"

ZIP_PATH = SDK_DIR / ZIP_NAME
SDK_FOLDER_PATH = SDK_DIR / SDK_FOLDER_NAME

print(f"[INFO] ZIP esperado: {ZIP_NAME}")
print(f"[INFO] Pasta local do SDK: {SDK_FOLDER_PATH}")

# ------------------------------------------------------------
# 1. Download do ZIP (se necessário)
# ------------------------------------------------------------
if not ZIP_PATH.exists():
    print(f"[INFO] Baixando SDK a partir de:\n{DOWNLOAD_URL}\n")
    with requests.get(DOWNLOAD_URL, stream=True) as r:
        r.raise_for_status()
        with open(ZIP_PATH, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
    print(f"[OK] Download concluído: {ZIP_PATH}")
else:
    print(f"[OK] ZIP já presente localmente: {ZIP_PATH}")

# ------------------------------------------------------------
# 2. Extração do SDK
# ------------------------------------------------------------
if not SDK_FOLDER_PATH.exists():
    print(f"[INFO] Extraindo SDK para: {SDK_FOLDER_PATH}")
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall(SDK_DIR)
    print("[OK] SDK extraído com sucesso.")
else:
    print(f"[OK] SDK já extraído em: {SDK_FOLDER_PATH}")

# ------------------------------------------------------------
# 3. Localizar o binário CIP
# ------------------------------------------------------------
CIP_BIN = SDK_FOLDER_PATH / "bin" / "cip"

if not CIP_BIN.exists():
    raise FileNotFoundError(f"[ERRO] Binário CIP não encontrado em: {CIP_BIN}")

# ------------------------------------------------------------
# 4. Garantir permissão de execução (Linux)
# ------------------------------------------------------------
st = os.stat(CIP_BIN)
os.chmod(CIP_BIN, st.st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)

print(f"[OK] Binário CIP pronto para uso em: {CIP_BIN}")


[INFO] ZIP esperado: cip-sdk-linux-idx0.zip
[INFO] Pasta local do SDK: /home/alvaro/descobrindo_caos/sdk_local/cip-sdk-linux-idx0
[INFO] Baixando SDK a partir de:
https://api.delta-cip.com.br/downloads/colab/cip-sdk-linux-idx0.zip

[OK] Download concluído: /home/alvaro/descobrindo_caos/sdk_local/cip-sdk-linux-idx0.zip
[INFO] Extraindo SDK para: /home/alvaro/descobrindo_caos/sdk_local/cip-sdk-linux-idx0
[OK] SDK extraído com sucesso.
[OK] Binário CIP pronto para uso em: /home/alvaro/descobrindo_caos/sdk_local/cip-sdk-linux-idx0/bin/cip


## CÉLULA 3 — Função utilitária `run_cip` e lógica de interoperabilidade

Nesta célula definimos um pequeno utilitário para executar o binário CIP local.

Pontos conceituais importantes:

* O **mesmo binário** CIP será usado em todas as verificações;

* As chaves foram geradas por **SDKs distintos**, com **IDX diferentes**;

* O binário local **não “adapta” seu comportamento** ao IDX da chave;

Toda a lógica de validação é dirigida **exclusivamente pela chave CIP-T**.

In [3]:
# ============================================================
# CÉLULA 3 — Utilitário para executar o binário CIP
# ============================================================

def run_cip(args):
    """
    Executa o binário CIP local com os argumentos fornecidos,
    medindo o tempo de execução e capturando stdout e stderr.

    Observação fundamental:
    ------------------------
    Este utilitário sempre invoca o MESMO binário CIP,
    independentemente da origem da chave utilizada na verificação.

    A interoperabilidade demonstrada neste notebook decorre
    exclusivamente da estrutura espectral descrita na chave CIP-T,
    e não de qualquer lógica condicional no binário local.
    """
    inicio = time.time()
    proc = subprocess.run(
        [str(CIP_BIN)] + args,
        cwd=SDK_FOLDER_PATH,
        capture_output=True,
        text=True,
        encoding="utf-8",
        errors="replace",
    )
    duracao = time.time() - inicio
    return proc, duracao

print("[OK] Função utilitária run_cip configurada.")

print(
    "\nNeste notebook, um ÚNICO binário CIP local será utilizado para:\n"
    "• verificar uma chave gerada em Linux (SDK IDX 18)\n"
    "• verificar uma chave gerada em Windows (SDK IDX 20)\n\n"
    "O binário não depende do IDX do SDK emissor da chave."
)

print(
    "A decisão de integridade é tomada exclusivamente com base em:\n"
    "• metadados da chave CIP-T,\n"
    "• matriz espectral embutida na chave,\n"
    "• e no conteúdo do arquivo verificado.\n\n"
    "Se o arquivo for idêntico, a integridade deve ser confirmada.\n"
    "Se houver qualquer alteração, a verificação falha — para TODAS as chaves."
)


[OK] Função utilitária run_cip configurada.

Neste notebook, um ÚNICO binário CIP local será utilizado para:
• verificar uma chave gerada em Linux (SDK IDX 18)
• verificar uma chave gerada em Windows (SDK IDX 20)

O binário não depende do IDX do SDK emissor da chave.
A decisão de integridade é tomada exclusivamente com base em:
• metadados da chave CIP-T,
• matriz espectral embutida na chave,
• e no conteúdo do arquivo verificado.

Se o arquivo for idêntico, a integridade deve ser confirmada.
Se houver qualquer alteração, a verificação falha — para TODAS as chaves.


## CÉLULA 4 — Download do arquivo de teste e importação das chaves externas (com progresso)

Nesta célula trazemos para o ambiente do notebook **os artefatos reais do experimento**:

* um **arquivo público grande** (NOAA GSOD 2023);

* duas **chaves CIP-T distintas**, geradas previamente:

  * uma em **Linux (IDX 18)**,

  * outra em **Windows (IDX 20)**.

Esses artefatos **não são gerados aqui**.  
Eles representam um cenário realista de uso do protocolo: SDKs já distribuídos e chaves já emitidas.

In [4]:
# ============================================================
# CÉLULA 4 — Download NOAA + Chaves (IDX 18 / IDX 20) com tqdm
# ============================================================

import requests
from tqdm import tqdm

# URLs públicas (artefatos reais do experimento)
BASE_URL = "https://delta-cip.com.br/downloads"

ARQ_REMOTE            = f"{BASE_URL}/noaa_gsod_2023.tar.gz"
CHAVE_LINUX_REMOTE   = f"{BASE_URL}/chave_cipt_assinado_idx18_N1024_20251124_115945.json"
CHAVE_WINDOWS_REMOTE = f"{BASE_URL}/chave_cipt_assinado_idx20_N1024_20251124_120307.json"

# Caminhos locais no notebook
ARQ_LOCAL            = ARQUIVOS_DIR / "noaa_gsod_2023.tar.gz"
CHAVE_LINUX_LOCAL    = CHAVES_DIR   / "chave_idx18_linux.json"
CHAVE_WINDOWS_LOCAL  = CHAVES_DIR   / "chave_idx20_windows.json"

# ------------------------------------------------------------
# Função utilitária com barra de progresso (tqdm)
# ------------------------------------------------------------
def baixar_com_progresso(url: str, destino: Path):
    """
    Faz download com barra de progresso.
    Se o arquivo já existir localmente e tiver tamanho > 0,
    o download é ignorado para garantir reprodutibilidade.
    """
    if destino.exists() and destino.stat().st_size > 0:
        print(
            f"[OK] Já existe localmente: {destino} "
            f"({destino.stat().st_size/1e6:.2f} MB)"
        )
        return

    print(f"[INFO] Baixando: {url}")

    with requests.get(url, stream=True) as r:
        r.raise_for_status()

        total = int(r.headers.get("content-length", 0))
        blocos = 8192

        barra = tqdm(
            total=total,
            unit="B",
            unit_scale=True,
            desc=destino.name
        )

        with open(destino, "wb") as f:
            for chunk in r.iter_content(chunk_size=blocos):
                if chunk:
                    f.write(chunk)
                    barra.update(len(chunk))

        barra.close()

    print(
        f"[OK] Download concluído: {destino} "
        f"({destino.stat().st_size/1e6:.2f} MB)"
    )

# ------------------------------------------------------------
# Download do arquivo de teste e das chaves externas
# ------------------------------------------------------------
baixar_com_progresso(ARQ_REMOTE, ARQ_LOCAL)
baixar_com_progresso(CHAVE_LINUX_REMOTE, CHAVE_LINUX_LOCAL)
baixar_com_progresso(CHAVE_WINDOWS_REMOTE, CHAVE_WINDOWS_LOCAL)

print("\n[OK] Artefatos disponíveis no ambiente local:")
print(f"  • Arquivo NOAA............: {ARQ_LOCAL}")
print(f"  • Chave Linux (IDX 18)....: {CHAVE_LINUX_LOCAL}")
print(f"  • Chave Windows (IDX 20)..: {CHAVE_WINDOWS_LOCAL}")

# ------------------------------------------------------------
# Verificação básica de sanidade (tamanho do arquivo)
# ------------------------------------------------------------
tamanho_bytes = ARQ_LOCAL.stat().st_size
print(f"\nTamanho do arquivo NOAA: {tamanho_bytes/1e6:.2f} MB")

if tamanho_bytes == 0 or tamanho_bytes > LIMITE_BYTES:
    raise RuntimeError(
        "Arquivo ausente, corrompido ou fora do limite permitido."
    )


[INFO] Baixando: https://delta-cip.com.br/downloads/noaa_gsod_2023.tar.gz


noaa_gsod_2023.tar.gz: 100%|███████████████████████████████████████| 109M/109M [05:11<00:00, 349kB/s]


[OK] Download concluído: /home/alvaro/descobrindo_caos/arquivos/noaa_gsod_2023.tar.gz (108.78 MB)
[INFO] Baixando: https://delta-cip.com.br/downloads/chave_cipt_assinado_idx18_N1024_20251124_115945.json


chave_idx18_linux.json: 100%|███████████████████████████████████| 88.0k/88.0k [00:00<00:00, 99.9kB/s]


[OK] Download concluído: /home/alvaro/descobrindo_caos/chaves_interop/chave_idx18_linux.json (0.09 MB)
[INFO] Baixando: https://delta-cip.com.br/downloads/chave_cipt_assinado_idx20_N1024_20251124_120307.json


chave_idx20_windows.json: 100%|█████████████████████████████████| 88.0k/88.0k [00:01<00:00, 76.1kB/s]

[OK] Download concluído: /home/alvaro/descobrindo_caos/chaves_interop/chave_idx20_windows.json (0.09 MB)

[OK] Artefatos disponíveis no ambiente local:
  • Arquivo NOAA............: /home/alvaro/descobrindo_caos/arquivos/noaa_gsod_2023.tar.gz
  • Chave Linux (IDX 18)....: /home/alvaro/descobrindo_caos/chaves_interop/chave_idx18_linux.json
  • Chave Windows (IDX 20)..: /home/alvaro/descobrindo_caos/chaves_interop/chave_idx20_windows.json

Tamanho do arquivo NOAA: 108.78 MB


## CÉLULA 5 — Inspeção estrutural das chaves CIP-T (IDX 18 / IDX 20)

Esta célula **não verifica integridade**.  
Ela apenas **expõe metadados essenciais** das duas chaves para deixar explícito que:

* ambas se referem ao **mesmo arquivo**;

* foram emitidas por **SDKs diferentes**;

* utilizam **IDX distintos**;

* carregam **estruturas espectrais próprias**.

Essas diferenças **não são um problema** — são justamente o que torna a interoperabilidade testável.

In [5]:
# ============================================================
# CÉLULA 5 — Inspeção das chaves IDX 18 (Linux) e IDX 20 (Windows)
# ============================================================

import json

def carregar_json(p: Path):
    with open(p, "r", encoding="utf-8") as f:
        return json.load(f)

def extrair_hash_integridade(chave: dict):
    """
    Extrai o hash de integridade principal de forma robusta,
    cobrindo variações entre versões de chave CIP-T.
    """
    candidatos = [
        "assinatura_integridade_final",
        "assinatura_integridade_chave_cipt",
    ]
    for c in candidatos:
        if c in chave and chave[c]:
            return chave[c]
    return None

def resumo_chave(chave: dict, titulo: str):
    print(f"[{titulo}]")
    print(f"  • SDK IDX usado........: {chave.get('sdk_idx_usado')}")
    print(f"  • Dimensão espectral N.: {chave.get('matrix_size_N_usado')}")
    print(f"  • Arquivo original.....: {chave.get('arquivo_original_nome')}")
    print(f"  • Operação.............: {chave.get('operacao')}")
    print(f"  • Timestamp............: {chave.get('timestamp_operacao_str')}")

    h = extrair_hash_integridade(chave)
    if h:
        print(f"  • Hash de integridade..: {h[:32]}...")
    else:
        print("  • Hash de integridade..: [não encontrado]")

    # presença da matriz espectral embutida (não exibimos o conteúdo)
    print(
        f"  • Matriz espectral P..: "
        f"{'presente' if chave.get('matriz_p_base64') else 'ausente'}"
    )
    print()

# ------------------------------------------------------------
# Carregar chaves previamente baixadas
# ------------------------------------------------------------
ch18 = carregar_json(CHAVE_LINUX_LOCAL)
ch20 = carregar_json(CHAVE_WINDOWS_LOCAL)

print("=== INSPEÇÃO DAS CHAVES — CIP-T (INTEROPERABILIDADE) ===\n")

resumo_chave(ch18, "Chave Linux — IDX 18")
resumo_chave(ch20, "Chave Windows — IDX 20")

print(
    "Observações importantes:\n"
    "• Ambas as chaves foram geradas sobre o MESMO arquivo.\n"
    "• Cada uma utiliza um IDX espectral distinto.\n"
    "• As estruturas internas (matrizes) são diferentes.\n\n"
    "Na próxima célula, usaremos UM ÚNICO binário CIP para verificar\n"
    "a integridade do arquivo usando AMBAS as chaves.\n"
    "Se ambas validarem, a interoperabilidade estrutural da CIP\n"
    "fica demonstrada de forma empírica e reprodutível."
)


=== INSPEÇÃO DAS CHAVES — CIP-T (INTEROPERABILIDADE) ===

[Chave Linux — IDX 18]
  • SDK IDX usado........: 18
  • Dimensão espectral N.: 1024
  • Arquivo original.....: noaa_gsod_2023.tar.gz
  • Operação.............: assinatura_total_cipt_v3_hibrido
  • Timestamp............: 20251124_115945
  • Hash de integridade..: 77f210ad293d2957a514f2a1e44fe2be...
  • Matriz espectral P..: presente

[Chave Windows — IDX 20]
  • SDK IDX usado........: 20
  • Dimensão espectral N.: 1024
  • Arquivo original.....: noaa_gsod_2023.tar.gz
  • Operação.............: assinatura_total_cipt_v3_hibrido
  • Timestamp............: 20251124_120307
  • Hash de integridade..: b660f961b8dee7fabc0cad8d3456a8f9...
  • Matriz espectral P..: presente

Observações importantes:
• Ambas as chaves foram geradas sobre o MESMO arquivo.
• Cada uma utiliza um IDX espectral distinto.
• As estruturas internas (matrizes) são diferentes.

Na próxima célula, usaremos UM ÚNICO binário CIP para verificar
a integridade do arquivo 

## CÉLULA 6 — Verificação Cruzada de Interoperabilidade

Ou seja:

Usar o binário local do notebook (Linux ou Windows, tanto faz — o ponto é: “um binário único”)

Para verificar:

* o arquivo NOAA com a chave Linux IDX 18

* o arquivo NOAA com a chave Windows IDX 20

A interoperabilidade só é provada se:

* **Ambas as verificações retornarem sucesso (returncode == 0)**,

* Mesmo cada chave tendo sido produzida em **outro sistema operacional**

* E com **Outro IDX**

Isso demonstra que:

> A CIP não depende do SO, nem do compilador, nem da máquina.  
> O que importa é a coerência espectral do arquivo com o IDX.  
> Se isso for mantido, qualquer binário válido deve verificar qualquer chave válida.

In [6]:
# ============================================================
# CÉLULA 6 — Verificação Cruzada de Interoperabilidade
# ============================================================

print("\n=== VERIFICAÇÃO CRUZADA — INTEROPERABILIDADE CIP ===\n")
print(f"[INFO] Binário CIP utilizado em todas as verificações:\n  {CIP_BIN}\n")

def verificar_cipt(arquivo: Path, chave_json: Path, descricao: str):
    """
    Executa o binário CIP LOCAL (nativo deste notebook)
    para verificar um arquivo usando uma chave CIP-T
    gerada em OUTRO ambiente computacional e com OUTRO IDX.

    Critério de sucesso:
      • retorno do processo == 0
    """
    print(f"\n--- {descricao} ---")

    proc, tempo = run_cip([
        "-t", "verificar",
        str(arquivo),
        str(chave_json)
    ])

    print("\n[STDOUT]\n")
    print(proc.stdout)

    if proc.returncode == 0:
        print(
            f"[OK] Verificação concluída com sucesso em {tempo:.3f} s.\n"
            "Resultado consistente com a definição do protocolo CIP."
        )
    else:
        print("\n[STDERR]\n")
        print(proc.stderr)
        raise RuntimeError(
            "FALHA DE INTEROPERABILIDADE:\n"
            "O binário CIP local não conseguiu validar uma chave CIP-T válida.\n"
            "Isso violaria a propriedade de interoperabilidade estrutural\n"
            "definida pelo protocolo."
        )

# ------------------------------------------------------------
# Execução das verificações cruzadas
# ------------------------------------------------------------

verificar_cipt(
    ARQ_LOCAL,
    CHAVE_LINUX_LOCAL,
    "Verificação com chave gerada em Linux (IDX 18)"
)

verificar_cipt(
    ARQ_LOCAL,
    CHAVE_WINDOWS_LOCAL,
    "Verificação com chave gerada em Windows (IDX 20)"
)

print("\n=== RESULTADO FINAL ===\n")
print(
    "As DUAS verificações foram realizadas com sucesso usando:\n"
    "• o MESMO binário CIP local;\n"
    "• chaves geradas em sistemas operacionais diferentes;\n"
    "• índices espectrais (IDX) distintos.\n\n"
    "Isso demonstra, de forma empírica e reprodutível, que:\n"
    "• a CIP não depende do sistema operacional;\n"
    "• não depende do compilador;\n"
    "• não depende do SDK específico que gerou a chave.\n\n"
    "A decisão de integridade depende exclusivamente da\n"
    "coerência espectral entre o arquivo e a estrutura descrita na chave CIP-T."
)



=== VERIFICAÇÃO CRUZADA — INTEROPERABILIDADE CIP ===

[INFO] Binário CIP utilizado em todas as verificações:
  /home/alvaro/descobrindo_caos/sdk_local/cip-sdk-linux-idx0/bin/cip


--- Verificação com chave gerada em Linux (IDX 18) ---

[STDOUT]

[INFO] OpenMP: 18 threads.
[INFO] Eigen: 18 threads.

════════════════[ CIP - Configuração SDK ]════════════════
[INFO] Idx do SDK determinado automaticamente: 0
[SUCESSO] Configuração do SDK (idx 0) carregada com sucesso.
[INFO] Idx da matriz de Assinatura (pré-calculada): 0 | Idx da matriz de Cifragem (pré-calculada): 1

════════════[ CIP - Verificação (MODO TOTAL) ]════════════
[INFO] Arquivo a ser verificado: noaa_gsod_2023.tar.gz
[INFO] Chave CIPT usada: chave_idx18_linux.json
[INFO] Verificando integridade da própria chave CIP-T...
[INFO] Hash esperado (assinatura_integridade_chave_cipt): 46a81b7ee6fafd983d764ab43e7d46f07f8a2806019adbd9a2a347e1566bd85b
[INFO] Hash calculado: 46a81b7ee6fafd983d764ab43e7d46f07f8a2806019adbd9a2a347e1566bd85

## CÉLULA 7 — Tabela comparativa das chaves (Linux IDX 18 vs Windows IDX 20)

In [7]:
# ============================================================
# CÉLULA 7 — Tabela comparativa das chaves (IDX 18 vs IDX 20)
# ============================================================

import pandas as pd

def trunc(h, n=16):
    if not h:
        return "[ausente]"
    return h[:n] + "..."

def linha_resumo(chave, origem):
    return {
        "origem": origem,
        "arquivo": chave.get("arquivo_original_nome"),
        "sdk_idx_usado": chave.get("sdk_idx_usado"),
        "N": chave.get("matrix_size_N_usado"),
        "operacao": chave.get("operacao"),
        "tamanho_bytes": chave.get("tamanho_original_bytes"),
        "timestamp": chave.get("timestamp_operacao_str"),
        "hash_integridade_chave": trunc(chave.get("assinatura_integridade_chave_cipt")),
        "hash_integridade_arquivo": trunc(chave.get("assinatura_integridade_final")),
    }

df_chaves = pd.DataFrame([
    linha_resumo(ch18, "Linux (SDK IDX 18)"),
    linha_resumo(ch20, "Windows (SDK IDX 20)")
]).sort_values("sdk_idx_usado").reset_index(drop=True)

display(df_chaves)

print(
    "\nINTERPRETAÇÃO:\n"
    "• Ambas as chaves referem-se ao MESMO arquivo e à MESMA dimensão N.\n"
    "• Os hashes de integridade do ARQUIVO são distintos porque cada SDK\n"
    "  utiliza uma base espectral própria (IDX diferente).\n"
    "• Apesar disso, ambas as chaves foram validadas com sucesso pelo\n"
    "  MESMO binário CIP neste notebook.\n\n"
    "Isso confirma que a interoperabilidade da CIP não depende do SDK que\n"
    "gerou a chave, mas exclusivamente da coerência espectral descrita nela."
)


,origem,arquivo,sdk_idx_usado,N,operacao,tamanho_bytes,timestamp,hash_integridade_chave,hash_integridade_arquivo
0,Linux (SDK IDX 18),noaa_gsod_2023.tar.gz,18,1024,assinatura_total_cipt_v3_hibrido,108776234,20251124_115945,46a81b7ee6fafd98...,77f210ad293d2957...
1,Windows (SDK IDX 20),noaa_gsod_2023.tar.gz,20,1024,assinatura_total_cipt_v3_hibrido,108776234,20251124_120307,550812b6bdd698b2...,b660f961b8dee7fa...



INTERPRETAÇÃO:
• Ambas as chaves referem-se ao MESMO arquivo e à MESMA dimensão N.
• Os hashes de integridade do ARQUIVO são distintos porque cada SDK
  utiliza uma base espectral própria (IDX diferente).
• Apesar disso, ambas as chaves foram validadas com sucesso pelo
  MESMO binário CIP neste notebook.

Isso confirma que a interoperabilidade da CIP não depende do SDK que
gerou a chave, mas exclusivamente da coerência espectral descrita nela.


## CÉLULA 8 — Corrupção de 1 bit + Verificação Cruzada (Linux idx18 e Windows idx20)

In [8]:
# ============================================================
# CÉLULA 8 — Corrupção de 1 bit + Verificação Cruzada
# ============================================================

import os
import random
import shutil
from pathlib import Path

print("\n=== TESTE CRUZADO — DETECÇÃO DE CORRUPÇÃO (1 BIT) ===")

# (Opcional) tornar experimento reprodutível
random.seed(42)

# ------------------------------------------------------------
# 1. Criar uma cópia do arquivo e corromper 1 bit
# ------------------------------------------------------------
arquivo_corrompido = ARQ_LOCAL.parent / f"{ARQ_LOCAL.stem}_1bit{ARQ_LOCAL.suffix}"
shutil.copy2(ARQ_LOCAL, arquivo_corrompido)

def flip_bit(path: Path):
    with open(path, "r+b") as f:
        f.seek(0, os.SEEK_END)
        tamanho = f.tell()
        if tamanho == 0:
            raise RuntimeError("Arquivo inválido para corrupção (tamanho zero).")

        pos = random.randint(0, tamanho - 1)
        bit = random.randint(0, 7)

        f.seek(pos)
        original = f.read(1)[0]

        novo = original ^ (1 << bit)
        f.seek(pos)
        f.write(bytes([novo]))

    return pos, bit, original, novo

pos, bit, antigo, novo = flip_bit(arquivo_corrompido)

print(
    f"[INFO] 1 bit corrompido em byte {pos}, bit {bit} "
    f"({antigo:02x} → {novo:02x})"
)

# ------------------------------------------------------------
# 2. Verificar com chave Linux (IDX 18)
# ------------------------------------------------------------
print("\n--- Verificando com chave Linux IDX 18 ---")

proc_lin, t_lin = run_cip([
    "-t", "verificar", str(arquivo_corrompido), str(CHAVE_LINUX_LOCAL)
])

print("\n[STDOUT]\n", proc_lin.stdout)

if proc_lin.returncode != 0:
    print(f"[OK] Chave Linux rejeitou o arquivo corrompido — {t_lin:.3f} s")
else:
    raise RuntimeError("ERRO CRÍTICO: chave Linux aceitou arquivo corrompido.")

# ------------------------------------------------------------
# 3. Verificar com chave Windows (IDX 20)
# ------------------------------------------------------------
print("\n--- Verificando com chave Windows IDX 20 ---")

proc_win, t_win = run_cip([
    "-t", "verificar", str(arquivo_corrompido), str(CHAVE_WINDOWS_LOCAL)
])

print("\n[STDOUT]\n", proc_win.stdout)

if proc_win.returncode != 0:
    print(f"[OK] Chave Windows rejeitou o arquivo corrompido — {t_win:.3f} s")
else:
    raise RuntimeError("ERRO CRÍTICO: chave Windows aceitou arquivo corrompido.")

# ------------------------------------------------------------
# 4. Resultado Final
# ------------------------------------------------------------
print("\n=== RESULTADO FINAL ===")
print(
    "O MESMO arquivo corrompido (1 bit) foi rejeitado por DUAS chaves\n"
    "geradas em ambientes distintos (Linux IDX 18, Windows IDX 20),\n"
    "utilizando o MESMO binário CIP local.\n\n"
    "A detecção não depende de plataforma, SDK ou compilador.\n"
    "Ela decorre exclusivamente da incoerência estrutural introduzida no conteúdo."
)



=== TESTE CRUZADO — DETECÇÃO DE CORRUPÇÃO (1 BIT) ===
[INFO] 1 bit corrompido em byte 85822412, bit 1 (07 → 05)

--- Verificando com chave Linux IDX 18 ---

[STDOUT]
 [INFO] OpenMP: 18 threads.
[INFO] Eigen: 18 threads.

════════════════[ CIP - Configuração SDK ]════════════════
[INFO] Idx do SDK determinado automaticamente: 0
[SUCESSO] Configuração do SDK (idx 0) carregada com sucesso.
[INFO] Idx da matriz de Assinatura (pré-calculada): 0 | Idx da matriz de Cifragem (pré-calculada): 1

════════════[ CIP - Verificação (MODO TOTAL) ]════════════
[INFO] Arquivo a ser verificado: noaa_gsod_2023.tar_1bit.gz
[INFO] Chave CIPT usada: chave_idx18_linux.json
[INFO] Verificando integridade da própria chave CIP-T...
[INFO] Hash esperado (assinatura_integridade_chave_cipt): 46a81b7ee6fafd983d764ab43e7d46f07f8a2806019adbd9a2a347e1566bd85b
[INFO] Hash calculado: 46a81b7ee6fafd983d764ab43e7d46f07f8a2806019adbd9a2a347e1566bd85b
[SUCESSO] Integridade da propria chave assinatura_integridade_chave_cipt

## Encerramento — Interoperabilidade Espectral da CIP

Este notebook documenta, de forma pública e reprodutível, que a Cifra de Integridade Primal (CIP):

1. **Assina o mesmo arquivo público em ambientes distintos**  
   - Uma chave foi gerada em Linux (SDK IDX 18).  
   - Outra chave foi gerada em Windows (SDK IDX 20).  
   - Ambas em modo TOTAL, sobre o mesmo arquivo NOAA (`noaa_gsod_2023.tar.gz`), com `N = 1024`.

2. **Verifica ambas as chaves com um único binário nativo**  
   O binário CIP local (executado neste notebook) validou:
   - a chave Linux (IDX 18) usando a matriz embutida na própria chave;
   - a chave Windows (IDX 20) usando a matriz correspondente ao seu IDX.

3. **Diferencia explicitamente o papel do IDX**  
   O campo `sdk_idx_usado` identifica a base harmônica associada à assinatura:
   - quando o IDX da chave coincide com o IDX local, a verificação utiliza a matriz pré-calculada do SDK;
   - quando o IDX da chave é externo, a verificação utiliza a matriz embutida na própria chave CIP-T.

   Em ambos os casos, o critério de aceitação é idêntico:  
   o hash de integridade final do arquivo deve coincidir exatamente com o registrado na chave.

4. **Evidencia independência de plataforma e ambiente**  
   O resultado da verificação não depende do sistema operacional, do compilador ou do ambiente onde a
   assinatura foi produzida. A decisão de integridade depende exclusivamente da coerência estrutural entre:
   - o conteúdo do arquivo,
   - a base harmônica (IDX),
   - e a estrutura matricial registrada na chave.

5. **Demonstra consistência estrutural sob verificação cruzada**  
   Um único binário CIP foi capaz de validar chaves produzidas em ambientes distintos e com IDX distintos,
   sem ajustes manuais ou parâmetros específicos por plataforma.

---

Do ponto de vista institucional, este experimento mostra que:

- é possível adotar um **critério único de integridade verificável** baseado em uma régua espectral comum;
- diferentes SDKs e plataformas podem coexistir sem comprometer a verificabilidade cruzada;
- qualquer instituição pode verificar, de forma independente, assinaturas produzidas em outro ambiente,
  desde que respeitada a mesma estrutura aritmética de base.

Os resultados apresentados neste notebook não dependem de hipóteses externas ou de confiança prévia
no ambiente de execução. Eles decorrem exclusivamente do comportamento observado do protocolo
sob verificação cruzada.
